In [11]:
import pandas as pd

In [12]:
pd.set_option('display.max_rows', 20)

In [13]:
rfm = pd.read_excel('RFMwithExcel.xlsx')
rfm1 = rfm[['CustomerID', 'SalesOrderNumber', 'SalesAmount', 'OrderDate']].copy()

In [36]:
# No missing values
rfm1.isna().mean()

CustomerID          0.0
SalesOrderNumber    0.0
SalesAmount         0.0
OrderDate           0.0
dtype: float64

In [24]:
# No duplicates
c[rfm1['SalesOrderNumber'].duplicated(keep=False)]

,CustomerID,SalesOrderNumber,SalesAmount,OrderDate


In [23]:
# Customer ID repeats
rfm1[rfm1['CustomerID'].duplicated(keep=False)].sort_values('CustomerID').head(10)

,CustomerID,SalesOrderNumber,SalesAmount,OrderDate
17377,11000,SO51522,2341.97,2013-01-18
16427,11000,SO43793,3399.99,2011-01-19
244,11000,SO57418,2507.03,2013-05-03
23580,11001,SO72773,588.96,2013-12-10
8705,11001,SO51493,2419.93,2013-01-16
1054,11001,SO43767,3374.99,2011-01-15
27448,11002,SO43736,3399.99,2011-01-07
3807,11002,SO53237,2419.06,2013-02-23
26016,11002,SO51238,2294.99,2012-12-31
22960,11003,SO51315,2318.96,2013-01-05


In [27]:
# OrderDate has replicates
rfm1.sort_values('OrderDate').head(10)

,CustomerID,SalesOrderNumber,SalesAmount,OrderDate
2531,25863,SO43699,3399.9900,2010-12-29
19985,11003,SO43701,3399.9900,2010-12-29
3651,14501,SO43700,699.0982,2010-12-29
27471,28389,SO43698,3399.9900,2010-12-29
27432,21768,SO43697,3578.2700,2010-12-29
13866,11005,SO43704,3374.9900,2010-12-30
14538,27645,SO43702,3578.2700,2010-12-30
19765,11011,SO43705,3399.9900,2010-12-30
24730,16624,SO43703,3578.2700,2010-12-30
17247,27621,SO43706,3578.2700,2010-12-31


In [34]:
rfm1.groupby(['CustomerID', 'SalesOrderNumber']).size()

CustomerID  SalesOrderNumber
11000       SO43793             1
            SO51522             1
            SO57418             1
11001       SO43767             1
            SO51493             1
            SO72773             1
11002       SO43736             1
            SO51238             1
            SO53237             1
11003       SO43701             1
                               ..
29474       SO44949             1
29475       SO45024             1
29476       SO45199             1
29477       SO60449             1
29478       SO60955             1
29479       SO49617             1
29480       SO62341             1
29481       SO45427             1
29482       SO49746             1
29483       SO49665             1
Length: 27659, dtype: int64

In [31]:
rfm1.groupby('OrderDate')['OrderDate'].count()

OrderDate
2010-12-29     5
2010-12-30     4
2010-12-31     5
2011-01-01     2
2011-01-02     5
2011-01-03     4
2011-01-04     3
2011-01-05     3
2011-01-06     6
2011-01-07     3
              ..
2014-01-19    32
2014-01-20    30
2014-01-21    40
2014-01-22    24
2014-01-23    32
2014-01-24    29
2014-01-25    32
2014-01-26    31
2014-01-27    23
2014-01-28    40
Name: OrderDate, Length: 1124, dtype: int64

In [72]:
# Recency
recency = rfm1.groupby('CustomerID')['OrderDate'].max().to_frame().reset_index()
# Frequency
frequency = rfm1.groupby('CustomerID')['SalesOrderNumber'].count().to_frame().reset_index()
# Monetary
monetary = rfm1.groupby('CustomerID')['SalesAmount'].sum().to_frame().reset_index()


In [79]:
from datetime import datetime as dt


datetime.datetime(2014, 2, 15, 0, 0)

In [110]:
rfm_df = recency.merge(frequency) \
            .merge(monetary)

(dt(2014, 2, 15) - rfm_df['OrderDate'])[0].days

288

In [111]:
rfm_df['DaysSinceLastOrder'] = (dt(2014, 2, 15) - rfm_df['OrderDate']).apply(lambda x: x.days)

In [112]:
rfm_df.head()

,CustomerID,OrderDate,SalesOrderNumber,SalesAmount,DaysSinceLastOrder
0,11000,2013-05-03,3,8248.99,288
1,11001,2013-12-10,3,6383.88,67
2,11002,2013-02-23,3,8114.04,357
3,11003,2013-05-10,3,8139.29,281
4,11004,2013-05-01,3,8196.01,290


In [124]:
rfm_df['Recency Score'] = 1 - rfm_df['DaysSinceLastOrder'].rank(pct=True)
rfm_df['Frequency Score'] = rfm_df['SalesOrderNumber'].rank(pct=True)
rfm_df['Monetary Score'] = rfm_df['SalesAmount'].rank(pct=True)

In [131]:
rfm_df['RFM Average'] = (rfm_df['Recency Score'] + rfm_df['Frequency Score'] + rfm_df['Monetary Score']) / 3
rfm_df.sort_values('RFM Average', ascending=False)

,CustomerID,OrderDate,SalesOrderNumber,SalesAmount,DaysSinceLastOrder,Recency Score,Frequency Score,Monetary Score,RFM Average
432,11432,2013-12-25,6,9330.1996,52,0.936783,0.997782,0.997565,0.977377
429,11429,2013-12-24,6,10468.3796,53,0.932996,0.997782,0.998485,0.976421
1314,12314,2013-12-28,4,7997.9196,49,0.948225,0.990830,0.987070,0.975375
1129,12129,2013-12-25,4,8557.4096,52,0.936783,0.990830,0.997295,0.974969
2580,13580,2013-12-27,4,7285.5700,50,0.945088,0.990830,0.986312,0.974077
2606,13606,2013-12-27,4,7195.5800,50,0.945088,0.990830,0.984473,0.973464
1302,12302,2013-12-26,4,8091.8896,51,0.940976,0.990830,0.988368,0.973391
425,11425,2013-12-21,6,10528.6282,56,0.921878,0.997782,0.998539,0.972733
2256,13256,2013-12-25,4,7494.8796,52,0.936783,0.990830,0.986691,0.971435
1330,12330,2013-12-24,4,7933.4096,53,0.932996,0.990830,0.986962,0.970263


In [ ]:
df_train.rank()

In [ ]:
df_train = pd.DataFrame({'Values': 1000*np.random.rand(15712)})

#Sort Data
df_train = df_train.sort_values('Values')

# Calculating Rank and Rank_Pct for demo purposes 
#but note that it is not needed for the solution
# The ranking of the validation data below does not depend on this
df_train['Rank'] = df_train.rank()
df_train['Rank_Pct']= df_train.Values.rank(pct=True)

# Demonstrate how Rank Percentile is calculated
# This gives the same value as .rank(pct=True)
pct_increment = 1./len(df_train)
df_train['Rank_Pct_Manual'] = df_train.Rank*pct_increment

df_train.head()

In [28]:
rfm1

,CustomerID,SalesOrderNumber,SalesAmount,OrderDate
0,15084,SO54147,2419.0600,2013-03-09
1,14522,SO66951,53.9900,2013-09-24
2,16190,SO72219,71.9700,2013-12-03
3,21225,SO47141,2181.5625,2012-02-06
4,12188,SO54173,2354.9800,2013-03-10
5,13314,SO58179,2410.9400,2013-05-17
6,16466,SO48708,2181.5625,2012-06-27
7,22191,SO58150,94.4600,2013-05-17
8,13185,SO58673,42.2800,2013-05-26
9,12978,SO61576,50.2300,2013-07-05
